This file will be used to train the policies and use them to generate the trajectories. 

Let's start by creating the environment we will be using: Cartpool

In [ ]:
import gym
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16, 10)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam
torch.manual_seed(0)

# for saving/loading
import os

import base64, io

# For visualization
from gym.wrappers.monitoring import video_recorder
from IPython.display import HTML
from IPython import display
import glob



In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
env = gym.make('CartPole-v0')
env.seed(0)

C:\Users\turin\AppData\Roaming\Python\Python311\site-packages\gym\envs\registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
C:\Users\turin\AppData\Roaming\Python\Python311\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\turin\AppData\Roaming\Python\Python311\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\turin\AppData\Roaming\Python\Python311\site-packages\gym\core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is mar

[0]

For the definition of classes and functions for training the policies, refer to train_policy_func.py file 

In [ ]:
from Train_policy_func import Policy, device
from Generate_traj_func import generate_trajectory

policy = Policy(state_size=4, action_size=2).to(device)
# optionally: policy.load_state_dict(torch.load("best_policy.pth"))

# Generate one trajectory: 
traj = generate_trajectory(policy, env)
print(f"Episode length: {len(traj)}, Total reward: {sum(s['reward'] for s in traj)}")



Episode length: 18, Total reward: 18.0


C:\Users\turin\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [ ]:
from PPO import ppo_update, RolloutBuffer

# Hyperparameters
env_name     = "CartPole-v0"
total_steps  = 10_000     # per update
updates      = 100        # how many times to run rollout+update
clip_eps     = 0.2
gamma, lam   = 0.99, 0.95
lr           = 3e-4
batch_size   = 64
epochs       = 4

# Instantiate
env       = gym.make(env_name)
policy    = Policy().to(device)
optimizer = optim.Adam(policy.parameters(), lr=lr)
buffer    = RolloutBuffer()

for update in range(updates):
    state = env.reset()
    steps = 0

    # 1) Roll out until we have total_steps
    while steps < total_steps:
        action, logp, value = policy.act(state)
        next_state, reward, done, _ = env.step(action)
        buffer.store(state, action, logp, reward, value, done)
        state = next_state
        steps += 1
        if done:
            state = env.reset()

    # 2) Compute last value (for GAE)
    _, last_value = policy(torch.from_numpy(state).float().unsqueeze(0).to(device))

    # 3) PPO update
    ppo_update(policy, optimizer, buffer, clip_eps, epochs, batch_size)

    # 4) (Optional) track progress
    if (update + 1) % 10 == 0:
        print(f"Update {update+1}/{updates} completed.")
